In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.api as smf
import scipy.stats as stat
from scipy.stats import norm
import math as ma

In [2]:
def prix_St_norm1(So,K,T,sigma,r,style):
  const=100000
  #browm1=  np.random.normal(0,1,const)
  St1=np.zeros((const))
  St2=np.zeros((const))
  for j in range(const):
    browm1=  np.random.normal(0,1)
    St1[j] = So*ma.exp((r-(sigma**2)/2)*T + sigma*ma.sqrt(T)*browm1)
    St2[j]= So*ma.exp((r-(sigma**2)/2)*T - sigma*ma.sqrt(T)*browm1)   
       
  payoff=np.ones(const)
  if (style=="call"):
    for i in range(const):
      payoff[i]=((max((St1[i]-K),0)+ (max((St2[i]-K),0)))*ma.exp(-r*T))/2
        
    return np.mean(payoff)
         
  if(style=='put'):
      for i in range(const):       
         payoff[i]=((max((K-St1[i]),0)+ (max((K-St2[i]),0)))*ma.exp(-r*T))/2
       
      return np.mean(payoff)
         

In [3]:
def prix_St_norm2(So,K,T,sigma,r,style):
  const=100000
  browm1=  np.random.normal(0,1,const)
  St1=np.zeros((const))
  St2=np.zeros((const))
  St1 = So*np.exp((r-(sigma**2)/2)*T + sigma*ma.sqrt(T)*browm1)
  St2 = So*np.exp((r-(sigma**2)/2)*T - sigma*ma.sqrt(T)*browm1)   
       
  payoff=np.ones(const)
  #Variable de controle
  Z=0.5*np.exp(-r*T)*(St1+St2)
  if (style=="call"):
    for i in range(const):
      payoff[i]=((max((St1[i]-K),0)+ (max((St2[i]-K),0)))*ma.exp(-r*T))/2  
    
         
  if(style=='put'):
      for i in range(const):       
         payoff[i]=((max((K-St1[i]),0)+ (max((K-St2[i]),0)))*ma.exp(-r*T))/2
      
  k=-np.cov(Z,payoff)/np.var(Z)

  variable_de_controle= payoff +k[0][1]*(Z-np.mean(Z))
  return np.mean(variable_de_controle)#, 1.96*np.std(variable_de_controle)/const

In [4]:

def qmc_simulation(M,d=1):
    qmc_= stat.qmc.Sobol(d, scramble=True) 
    x_sobol= qmc_.random_base2(M)
    return stat.norm.ppf(x_sobol)


def prix_St_norm(So,K,T,sigma,r,style):
  const=len(qmc_simulation(12))
  browm1=  qmc_simulation(12)
  St1=np.zeros((const))
  St2=np.zeros((const))
  St1 = So*np.exp((r-(sigma**2)/2)*T + sigma*ma.sqrt(T)*browm1)
  St2 = So*np.exp((r-(sigma**2)/2)*T - sigma*ma.sqrt(T)*browm1)       
  payoff=np.ones(const)
  #Variable de controle
  Z=0.5*np.exp(-r*T)*(St1+St2)
  if (style=="call"):
    for i in range(const):
      payoff[i]=((max((St1[i]-K),0)+ (max((St2[i]-K),0)))*ma.exp(-r*T))/2       
  if(style=='put'):
      for i in range(const):       
         payoff[i]=((max((K-St1[i]),0)+ (max((K-St2[i]),0)))*ma.exp(-r*T))/2    
  k=-np.cov(Z.T,payoff)/np.var(Z)
  variable_de_controle= payoff +k[0][1]*(Z-np.mean(Z))
  return np.mean(variable_de_controle)#, 1.96*np.std(variable_de_controle)/const, const

In [5]:
"""l=(prix_St_norm(100,110,1,0.25,0.05,'call'))
print(" prix de l'option :"+str(l[0])+" un interval de confince de "+str(l[1])+" pour "+str(l[2])+" Simulations")"""

'l=(prix_St_norm(100,110,1,0.25,0.05,\'call\'))\nprint(" prix de l\'option :"+str(l[0])+" un interval de confince de "+str(l[1])+" pour "+str(l[2])+" Simulations")'

In [6]:
def delta(So,K,T,sigma,r,style):
  h=1
  return (prix_St_norm(So-2*h,K,T,sigma,r,style)- 8*(prix_St_norm(So-h,K,T,sigma,r,style)) +8*prix_St_norm(So+h,K,T,sigma,r,style)-prix_St_norm(So+2*h,K,T,sigma,r,style))/(12*h)
#  (u(x-2h)-8*u(x-h)+8*u(x+h)+u(x+2h))/12h

def gamma(So,K,T,sigma,r,style):
  h=1
  return (delta(So-2*h,K,T,sigma,r,style)- 8*(delta(So-h,K,T,sigma,r,style)) +8*delta(So+h,K,T,sigma,r,style)-delta(So+h,K,T,sigma,r,style))/(12*h)

def theta(So,K,T,sigma,r,style):
  t=T/100
  return (prix_St_norm(So,K,T-2*t,sigma,r,style)- 8*(prix_St_norm(So,K,T-t,sigma,r,style)) +8*prix_St_norm(So,K,T+t,sigma,r,style)-prix_St_norm(So,K,T+2*t,sigma,r,style))/(12*t)
def rho(So,K,T,sigma,r,style):
  
  h=0.001
  return (prix_St_norm(So,K,T,sigma,r-2*h,style) - 8*(prix_St_norm(So,K,T,sigma,r-h,style)) + 8*prix_St_norm(So,K,T,sigma,r+h,style) - prix_St_norm(So,K,T,sigma,r+2*h,style))/(12*h)

def vega(So,K,T,sigma,r,style):
  h=sigma/100
  return (prix_St_norm(So,K,T,sigma-2*h,r,style)- 8*(prix_St_norm(So,K,T,sigma-h,r,style)) +8*prix_St_norm(So,K,T,sigma+h,r,style)-prix_St_norm(So,K,T,sigma+2*h,r,style))/(12*h)


In [7]:
def delta_barrier(So,K,T,sigma,r,style):
  h=1
  return (prix_St_norm(So-2*h,K,T,sigma,r,style)- 8*(prix_St_norm(So-h,K,T,sigma,r,style)) +8*prix_St_norm(So+h,K,T,sigma,r,style)-prix_St_norm(So+2*h,K,T,sigma,r,style))/(12*h)
#  (u(x-2h)-8*u(x-h)+8*u(x+h)+u(x+2h))/12h

def gamma_barrier(So,K,T,sigma,r,style):
  h=1
  return (delta(So-2*h,K,T,sigma,r,style)- 8*(delta(So-h,K,T,sigma,r,style)) +8*delta(So+h,K,T,sigma,r,style)-delta(So+h,K,T,sigma,r,style))/(12*h)

def theta_barrier(So,K,T,sigma,r,style):
  t=T/100
  return (prix_St_norm(So,K,T-2*t,sigma,r,style)- 8*(prix_St_norm(So,K,T-t,sigma,r,style)) +8*prix_St_norm(So,K,T+t,sigma,r,style)-prix_St_norm(So,K,T+2*t,sigma,r,style))/(12*t)
def rho_barrier(So,K,T,sigma,r,style):
  
  h=0.001
  return (prix_St_norm(So,K,T,sigma,r-2*h,style) - 8*(prix_St_norm(So,K,T,sigma,r-h,style)) + 8*prix_St_norm(So,K,T,sigma,r+h,style) - prix_St_norm(So,K,T,sigma,r+2*h,style))/(12*h)

def vega_barrier(So,K,T,sigma,r,style):
  h=sigma/100
  return (prix_St_norm(So,K,T,sigma-2*h,r,style)- 8*(prix_St_norm(So,K,T,sigma-h,r,style)) +8*prix_St_norm(So,K,T,sigma+h,r,style)-prix_St_norm(So,K,T,sigma+2*h,r,style))/(12*h)


In [8]:
print(delta(100,110,1,0.25,0.05,'call'))
print(gamma(100,110,1,0.25,0.05,'call'))
print(vega(100,110,1,0.25,0.05,'call'))
print(rho(100,110,1,0.25,0.05,'call'))
print(theta(100,110,1,0.25,0.05,'call'))

0.4776880794167087
0.013689808633150058
39.59416490047699
42.65563356730399
6.541715797926159


In [9]:

def qmc_simulation(M,d=1):
    qmc_= stat.qmc.Sobol(d, scramble=True) 
    x_sobol= qmc_.random_base2(M)
    return stat.norm.ppf(x_sobol)

qmc_simulation(10).shape

(1024, 1)

In [10]:
def qmc_plot(M):
    qmc_= stat.qmc.Sobol(d=2, scramble=True) 
    return qmc_.random_base2(M)

X=qmc_plot(10).T

#plt.scatter(X[1],X[0])


In [11]:
Y=np.random.random(size=(2**10,2)).T
#plt.scatter(Y[0],Y[1])

In [12]:
def var_montecarlo_europene(So,K,To,sigma,r,style,level=0.05):
  const=100000
  browm1=  np.random.normal(0,1,const)
  St1=np.zeros((const))
  St2=np.zeros((const))
  T= 14/365 
  St1 = So*np.exp((r-(sigma**2)/2)*T + sigma*ma.sqrt(T)*browm1)
  St2 = So*np.exp((r-(sigma**2)/2)*T - sigma*ma.sqrt(T)*browm1)   
  T= T/365    
  payoff=np.ones(const)
  Z=0.5*np.exp(-r*T)*(St1+St2)
  if (style=="call"):
    for i in range(const):
      payoff[i]=((max((St1[i]-K*ma.exp(-r*(To-T))),0)+ (max((St2[i]-K*ma.exp(-r*(To-T))),0)))*ma.exp(-r*T))/2  
  if(style=='put'):
      for i in range(const):       
         payoff[i]=((max((K*ma.exp(-r*(To-T))-St1[i]),0)+ (max((K*ma.exp(-r*(To-T))-St2[i]),0)))*ma.exp(-r*T))/2
  k=-np.cov(Z.T,payoff)/np.var(Z)

  vc= payoff +k[0][1]*(Z-np.mean(Z))   
  var=np.percentile(vc, level)
  cvar=pd.DataFrame(vc)[pd.DataFrame(vc)<= var].mean() 
  return var,cvar

In [13]:
d=var_montecarlo_europene(100,110,1,0.25,0.05,'call')
var_montecarlo_europene(917.8,925,57/365,0.015,0.0045205,'call')
d

(-0.6360688287507431,
 0   -1.269841
 dtype: float64)

In [14]:
def delta_BS(S,K,T,r,sigma, style):
    d1=(ma.log(S/K)+(r+(sigma**2)/2)*T)/(sigma*ma.sqrt(T))
    if(style=='call'):
       return norm.cdf(d1)
    if(style=='put'):
       return norm.cdf(d1)-1

def gamma_BS(S,K,T,r,sigma,style):
    d1=(ma.log(S/K)+(r+(sigma**2)/2)*T)/(sigma*ma.sqrt(T))
    return (norm.pdf(d1)/(S*sigma*ma.sqrt(T)))

def theta_BS(S,K,T,r,sigma,style):
    d1=(ma.log(S/K)+(r+(sigma**2)/2)*T)/(sigma*ma.sqrt(T))
    d2=d1-sigma*ma.sqrt(T)
    if(style=='call'):
      return ((-(S*sigma*norm.pdf(d1))/(2*ma.sqrt(T)))-K*r*ma.exp(-r*T)*norm.cdf(d2))
    if(style=='put'):
      return (-(S*sigma*norm.pdf(d1))/(2*ma.sqrt(T))+K*r*ma.exp(-r*T)*norm.cdf(-d2))
    
theta_BS(100,110,1,0.25,0.05,'call')

-21.401709882546307

In [15]:
def delta_gamma(So,K,To,sigma,r,style, alpha=0.05):
  d=delta_BS(So,K,To,r,sigma,style)
  g=gamma_BS(So,K,To,r,sigma,style)
  t=theta_BS(So,K,To,r,sigma,style)
  const=len(qmc_simulation(17))
  browm1=  qmc_simulation(17)
  St1=np.zeros((const))
  T= 1.4/365 
  St1 = So- So*np.exp((r-(sigma**2)/2)*T + sigma*ma.sqrt(T)*browm1)
  delta_c= St1*d +0.5*(St1**2)*g + T*t
  var=np.percentile(delta_c, 1-alpha)
  cvar=pd.DataFrame(delta_c)[pd.DataFrame(delta_c)<= var].mean().to_numpy()
  return var, cvar


delta_gamma(100,110,1,0.25,0.05,'call')

(-1.6869913168426742, array([-1.91017225]))

In [25]:

def option_asiatique(So,K,T,sigma,r,style):
  const=len(qmc_simulation(12))
  browm1=  qmc_simulation(12)
  St1=np.zeros((const))
  St2=np.zeros((const))
  dis=500
  t=np.arange(0,T+T/dis, T/dis)
  for i in range(dis+1):
    St1 = St1 + So*np.exp((r-(sigma**2)/2)*t[i] + sigma*ma.sqrt(t[i])*browm1)
    St2 = St2 + So*np.exp((r-(sigma**2)/2)*t[i] - sigma*ma.sqrt(t[i])*browm1) 
  St1=St1[:,0]/dis   
  St2=St2[:,0]/dis      
  payoff=np.ones(const)
  #Variable de controle
  Z=0.5*np.exp(-r*T)*(St1+St2)
  if (style=="call"):
    for i in range(const):
      payoff[i]=((max((St1[i]-K),0)+ (max((St2[i]-K),0)))*ma.exp(-r*T))/2       
  if(style=='put'):
      for i in range(const):       
         payoff[i]=((max((K-St1[i]),0)+ (max((K-St2[i]),0)))*ma.exp(-r*T))/2    
  k=-np.cov(Z.T,payoff)/np.var(Z)
  variable_de_controle= payoff +k[0][1]*(Z-np.mean(Z))
  return  np.mean(variable_de_controle)


def delta_asiatique(So,K,T,sigma,r,style):
  h=1
  return (option_asiatique(So-2*h,K,T,sigma,r,style)- 8*(option_asiatique(So-h,K,T,sigma,r,style)) +8*option_asiatique(So+h,K,T,sigma,r,style)-option_asiatique(So+2*h,K,T,sigma,r,style))/(12*h)
#  (u(x-2h)-8*u(x-h)+8*u(x+h)+u(x+2h))/12h

def gamma_asiatique(So,K,T,sigma,r,style):
  h=1
  return (delta_asiatique(So-2*h,K,T,sigma,r,style)- 8*(delta_asiatique(So-h,K,T,sigma,r,style)) +8*delta_asiatique(So+h,K,T,sigma,r,style)-delta_asiatique(So+h,K,T,sigma,r,style))/(12*h)

def theta_asiatique(So,K,T,sigma,r,style):
  t=T/100
  return (option_asiatique(So,K,T-2*t,sigma,r,style)- 8*(option_asiatique(So,K,T-t,sigma,r,style)) +8*option_asiatique(So,K,T+t,sigma,r,style)-option_asiatique(So,K,T+2*t,sigma,r,style))/(12*t)
def rho_asiatique(So,K,T,sigma,r,style):
  
  h=0.001
  return (option_asiatique(So,K,T,sigma,r-2*h,style) - 8*(option_asiatique(So,K,T,sigma,r-h,style)) + 8*option_asiatique(So,K,T,sigma,r+h,style) - option_asiatique(So,K,T,sigma,r+2*h,style))/(12*h)

def vega_asiatique(So,K,T,sigma,r,style):
  h=sigma/100
  return (option_asiatique(So,K,T,sigma-2*h,r,style)- 8*(option_asiatique(So,K,T,sigma-h,r,style)) +8*option_asiatique(So,K,T,sigma+h,r,style)-option_asiatique(So,K,T,sigma+2*h,r,style))/(12*h)


def delta_gamma_asiatique(So,K,To,sigma,r,style, alpha=0.05):
  d=delta_asiatique(So,K,To,r,sigma,style)
  g=gamma_asiatique(So,K,To,r,sigma,style)
  theta=theta_asiatique(So,K,To,r,sigma,style)
  const=len(qmc_simulation(8))
  browm1=  qmc_simulation(8)
  St1=np.zeros((const))
  T= 1.4/365 
  dis=100
  t=np.arange(0,T+T/dis, T/dis)
  for i in range(dis+1):
    St1 = St1 + So*np.exp((r-(sigma**2)/2)*t[i] + sigma*ma.sqrt(t[i])*browm1)
  St1=St1[:,0]/dis
  St1 = So- St1
  delta_c= St1*d +0.5*(St1**2)*g + T*theta
  var=np.percentile(delta_c, 1-alpha)
  cvar=pd.DataFrame(delta_c)[pd.DataFrame(delta_c)<= var].mean().to_numpy()
  return var, cvar




In [26]:
(option_asiatique(100,100,1,0.3,0.02,'call'),)
#delta_gamma_asiatique(100,110,1,0.25,0.05,'call')
#delta_asiatique(100,110,1,0.25,0.05,'call')

(8.457835957827402,)

In [18]:
def option_lookback1(So,K,T,sigma,r,style):
  const=len(qmc_simulation(14))
  browm1=  qmc_simulation(14)
  dis=300
  St1=np.zeros((const,dis+1))
  St2=np.zeros((const, dis+1))
 
  t=np.arange(0,T+T/dis, T/dis)
  for i in range(dis+1):
    St1[:,i:i+1] = So*np.exp((r-(sigma**2)/2)*t[i] + sigma*ma.sqrt(t[i])*browm1)
    St2[:,i:i+1]= So*np.exp((r-(sigma**2)/2)*t[i] - sigma*ma.sqrt(t[i])*browm1)  
  if (style=="call"):
      payoff=((max((np.amax(St1)-K),0)+ (max((np.amax(St1)-K),0)))*ma.exp(-r*T))/2       
  if(style=='put'): 
      payoff=((max((K-np.amin(St1)),0)+ (max(min(K-np.ammin(St2)),0)))*ma.exp(-r*T))/2    
  return  payoff

def option_lookback(So,K,T,sigma,r,style):
  const=len(qmc_simulation(14))
  browm1=  qmc_simulation(14)
  dis=300
  St1=np.zeros((const,dis+1))
  St2=np.zeros((const, dis+1))
 
  t=np.arange(0,T+T/dis, T/dis)
  for i in range(dis+1):
    St1[:,i:i+1] = So*np.exp((r-(sigma**2)/2)*t[i] + sigma*ma.sqrt(t[i])*browm1)
    St2[:,i:i+1]= So*np.exp((r-(sigma**2)/2)*t[i] - sigma*ma.sqrt(t[i])*browm1)  
  
  St1=np.mean(St1, axis=1)
  St2=np.mean(St2, axis=1)
  if (style=="call"):
      payoff=((max((np.amax(St1)-K),0)+ (max((np.amax(St1)-K),0)))*ma.exp(-r*T))/2       
  if(style=='put'): 
      payoff=((max((K-np.amin(St1)),0)+ (max(min(K-np.ammin(St2)),0)))*ma.exp(-r*T))/2    
  return  payoff

def delta_lookback(So,K,T,sigma,r,style):
  h=1
  return (option_lookback(So-2*h,K,T,sigma,r,style)- 8*(option_lookback(So-h,K,T,sigma,r,style)) +8*option_lookback(So+h,K,T,sigma,r,style)-option_lookback(So+2*h,K,T,sigma,r,style))/(12*h)
#  (u(x-2h)-8*u(x-h)+8*u(x+h)+u(x+2h))/12h

def gamma_lookback(So,K,T,sigma,r,style):
  h=1
  return (delta_lookback(So-2*h,K,T,sigma,r,style)- 8*(delta_lookback(So-h,K,T,sigma,r,style)) +8*delta_lookback(So+h,K,T,sigma,r,style)-delta_lookback(So+h,K,T,sigma,r,style))/(12*h)

def theta_lookback(So,K,T,sigma,r,style):
  t=T/100
  return (option_lookback(So,K,T-2*t,sigma,r,style)- 8*(option_lookback(So,K,T-t,sigma,r,style)) +8*option_lookback(So,K,T+t,sigma,r,style)-option_lookback(So,K,T+2*t,sigma,r,style))/(12*t)
def rho_lookback(So,K,T,sigma,r,style):
  
  h=0.001
  return (option_lookback(So,K,T,sigma,r-2*h,style) - 8*(option_lookback(So,K,T,sigma,r-h,style)) + 8*option_lookback(So,K,T,sigma,r+h,style) - option_lookback(So,K,T,sigma,r+2*h,style))/(12*h)

def vega_lookback(So,K,T,sigma,r,style):
  h=sigma/100
  return (option_lookback(So,K,T,sigma-2*h,r,style)- 8*(option_lookback(So,K,T,sigma-h,r,style)) +8*option_lookback(So,K,T,sigma+h,r,style)-option_lookback(So,K,T,sigma+2*h,r,style))/(12*h)
def delta_gamma_lookback(So,K,To,sigma,r,style, alpha=0.05):
  d=delta_lookback(So,K,To,r,sigma,style)
  g=gamma_lookback(So,K,To,r,sigma,style)
  theta=theta_lookback(So,K,To,r,sigma,style)
  const=len(qmc_simulation(15))
  browm1=  qmc_simulation(15)
  T= 1.4/365 
  dis=100
  St1=np.zeros((const, dis+1))

  t=np.arange(0,T+T/dis, T/dis)
  for i in range(dis+1):
    St1[:,i:i+1] = So*np.exp((r-(sigma**2)/2)*t[i] + sigma*ma.sqrt(t[i])*browm1)
  St1 =  np.max(St1,axis=1 )-So
  delta_c= St1*d +0.5*(St1**2)*g + T*theta
  var=np.percentile(delta_c, 1-alpha)
  cvar=pd.DataFrame(delta_c)[pd.DataFrame(delta_c)<= var].mean().to_numpy()
  return var, cvar

option_lookback(100,110,1,0.25,0.05,'call')
#delta_gamma_lookback(100,110,1,0.25,0.05,'call')

88.7987974499605

In [19]:
def option_barrierr(So,K,H,T,sigma,r,style):
  const=len(qmc_simulation(12))
  browm1=  qmc_simulation(12)
  dis=50
  St1=np.zeros((const,dis+1))
  St2=np.zeros((const, dis+1))
  t=np.arange(0,T+T/dis, T/dis)
  for i in range(dis+1):
    St1[:,i:i+1] = So*np.exp((r-(sigma**2)/2)*t[i] + sigma*ma.sqrt(t[i])*browm1)
    St2[:,i:i+1]= So*np.exp((r-(sigma**2)/2)*t[i] - sigma*ma.sqrt(t[i])*browm1)       
  payoff=np.ones(const)
  
  if (np.amax(St1)< H) and (np.amax(St2)< H):
    Z=0.5*np.exp(-r*T)*(St1[:,-1]-+St2[:,-1])
    if (style=="call"):
     for i in range(const):
      payoff[i]=((max((St1[i][-1]-K),0)+ (max((St2[i][-1]-K),0)))*ma.exp(-r*T))/2       
    if(style=='put'):
      for i in range(const):       
         payoff[i]=((max((K-St1[i][-1]),0)+ (max((K-St2[i][-1]),0)))*ma.exp(-r*T))/2    
    k=-np.cov(Z.T,payoff)/np.var(Z)
    variable_de_controle= payoff +k[0][1]*(Z-np.mean(Z))
    return np.mean(variable_de_controle)#, 1.96*np.std(variable_de_controle)/const, const
  
  return 0

def option_barrier(So,K,H,T,sigma,r,style):
  const=len(qmc_simulation(12))
  browm1=  qmc_simulation(12)
  dis=100
  St1=np.zeros((const,dis+1))
  St2=np.zeros((const, dis+1))
  t=np.arange(0,T+T/dis, T/dis)
  for i in range(dis+1):
    St1[:,i:i+1] = So*np.exp((r-(sigma**2)/2)*t[i] + sigma*ma.sqrt(t[i])*browm1)
    St2[:,i:i+1]= So*np.exp((r-(sigma**2)/2)*t[i] - sigma*ma.sqrt(t[i])*browm1)       
  
  St1=np.mean(St1, axis=1)
  St2=np.mean(St2, axis=1)
  print(np.amax(St1),np.amax(St2))
  if (np.amax(St1)< H) and (np.amax(St2)< H):
    return prix_St_norm(So,K,T,sigma,r,style)
  return 0

def delta_gamma_barrier(So,K,H,T,sigma,r,style, alpha=0.05):
  if option_barrier(So,K,H,T,sigma,r,style) :
    return delta_gamma(So,K,T,sigma,r,style, alpha=0.05)
  else:
    return 0


option_barrier(100,110,2000,1,0.25,0.05,'call')
delta_gamma_barrier(100,110,200,1,0.25,0.05,'call')

214.71810869032518 194.74159677748483
185.09688756581107 202.2300636573508


0

PRICING DES OPTIONS AVEC LA VOLATILITE SCTHOCHASTIQUE

In [20]:
"""with st.form("Prix de l'option"):
   st.write("Simulation du prix du sous-jacent")
   c_p=st.selectbox("Style de l'option", ["call","put"])
   So=st.number_input("Le prix initial de votre Sous-jacent",step=1)
   K=st.number_input("Le Strike",step=1)
   T=st.number_input("Temps en année",step=0.05)
   sigma=st.number_input("La volatilité",step=0.05)
   r=st.number_input("Taux sans risque",step=0.05)
   st.write("Parametre de Simulation")
   const=st.slider("Nombre de simulation",0,10000,step=10)
   alpha=st.number_input(" Niveau de risque",step=0.01, value=0.05)
   N=1000
   submitted = st.form_submit_button("Submit")
   if (submitted):
      st.write("Evolution possible du prix du sous-jacent N=100")
      fig, ax = plt.subplots()
      ax= plot_St(const,So,T,sigma,r)
      st.pyplot(fig)
      st.write("La valeur de l'option")
      st.write(prix_St_norm(const,So,K,T,sigma,r,c_p,alpha))
      st.write("La valeur de l'option par le modéle de black and Scholes: ",black_scholes(So,K,T,r,sigma,c_p))"""

'with st.form("Prix de l\'option"):\n   st.write("Simulation du prix du sous-jacent")\n   c_p=st.selectbox("Style de l\'option", ["call","put"])\n   So=st.number_input("Le prix initial de votre Sous-jacent",step=1)\n   K=st.number_input("Le Strike",step=1)\n   T=st.number_input("Temps en année",step=0.05)\n   sigma=st.number_input("La volatilité",step=0.05)\n   r=st.number_input("Taux sans risque",step=0.05)\n   st.write("Parametre de Simulation")\n   const=st.slider("Nombre de simulation",0,10000,step=10)\n   alpha=st.number_input(" Niveau de risque",step=0.01, value=0.05)\n   N=1000\n   submitted = st.form_submit_button("Submit")\n   if (submitted):\n      st.write("Evolution possible du prix du sous-jacent N=100")\n      fig, ax = plt.subplots()\n      ax= plot_St(const,So,T,sigma,r)\n      st.pyplot(fig)\n      st.write("La valeur de l\'option")\n      st.write(prix_St_norm(const,So,K,T,sigma,r,c_p,alpha))\n      st.write("La valeur de l\'option par le modéle de black and Scholes: 